# Web Scraping and Machine Learning in Python

The following provides an example of how basic web scraping and in Python
<ul>
<li>IPython Notebooks</li>
<li>Parsing HTML using [lxml](http://lxml.de/)</li>
<li>Data manipulation w/ [pandas](http://pandas.pydata.org)</li>
<li>Machine learning for classification: Multinomial Naive Bayes w/ [sklearn](http://scikit-learn.org)</li>
</ul>

Now lets see if we can teach our computer how to tell the difference between a Wall Street Journal headline and a Gawker headline. 

Inspiration for this notebook/presentation was provided by [this](http://nbviewer.ipython.org/github/nealcaren/workshop_2014/blob/master/notebooks/6_Classification.ipynb) and [this](http://nbviewer.ipython.org/github/cs109/content/blob/master/HW3_solutions.ipynb). If you want to run this notebook, or one like it, it'll be helpful for you to check out [Anaconda](https://store.continuum.io/cshop/anaconda/)


## Get the Data!
If we want to build a model with a discerning taste for quality news, we're going to have to find him some examples. Lucky for us, Gawker and the Wall Street Journal both have websites. 
### To the internet (with dev tools)!
#### [Gawker](http://gawker.com)
After poking around the gawker homepage with the help of command + option + i (mac) and a Chrome plugin, [XPath Helper](https://chrome.google.com/webstore/detail/xpath-helper/hgimnogjllphhhkhlmebbmlgjoejdpjl?hl=en), we can see that all the article titles can be grabbed with the xpath, `"//header/h1/a/text()"`. lxml lets us retrieve all the text that matches our xpath

In [1]:
from lxml import html 
x = html.parse('http://gawker.com')
titles = x.xpath('//header/h1/a/text()')
print "We got {} titles. Here are the first 5:".format(len(titles))
titles[:5]

We got 22 titles. Here are the first 5:


['April 19 Has Become Everyone-is-a-Threat Day',
 'BuzzFeed Deleted Posts Under Pressure from Its Own Business Department',
 "Texas Vet Fired After Posting About Cat She'd Allegedly Killed",
 'Jon Stewart on ',
 ': "I Think of Us as Turd Miners"']

This page only has 20 titles on it. If we want to properly train our model, we're going to need more examples. There's a "More Stories" button at the bottom of the page, which brings us to another, similarly structured page with new titles and another "More Stories" button. To get more examples, we'll repeat the process above in a loop with each successive iteration hitting the page pointed to by the "More Stories" button. In order to figure out what the link is, go do some more investigating! You can also just look at the code below. 

In [2]:
headlines = x.xpath('//header/h1/a/text()')

In [3]:
# These are the xpaths we determined from snooping 
next_button_xpath = "//div[@class='row load-more']//a[contains(@href, 'startTime')]/@href"
headline_xpath = '//header/h1/a/text()'

# We'll use sleep to add some time in between requests
# so that we're not bombarding Gawker's server too hard. 
from time import sleep

# Now we'll fill this list of gawker titles by starting
# at the landing page and following "More Stories" links
gawker_titles = []
base_url = 'http://gawker.com/{}'
next_page = "http://gawker.com/"
while len(gawker_titles) < 2000 and next_page:
    dom = html.parse(next_page)
    headlines = dom.xpath(headline_xpath)
    print "Retrieved {} titles from url: {}".format(len(headlines), next_page)
    gawker_titles += headlines
    next_pages = dom.xpath(next_button_xpath)
    if next_pages: 
        next_page = base_url.format(next_pages[0]) 
    else:
        print "No next button found"
        next_page = None
    sleep(5)

Retrieved 22 titles from url: http://gawker.com/
Retrieved 20 titles from url: http://gawker.com/?startTime=1429306066246
Retrieved 19 titles from url: http://gawker.com/?startTime=1429280486922
Retrieved 24 titles from url: http://gawker.com/?startTime=1429209060638
Retrieved 23 titles from url: http://gawker.com/?startTime=1429190460993
Retrieved 21 titles from url: http://gawker.com/?startTime=1429127220147
Retrieved 19 titles from url: http://gawker.com/?startTime=1429106400436
Retrieved 22 titles from url: http://gawker.com/?startTime=1429048200300
Retrieved 19 titles from url: http://gawker.com/?startTime=1429026600621
Retrieved 21 titles from url: http://gawker.com/?startTime=1428959400965
Retrieved 24 titles from url: http://gawker.com/?startTime=1428936300788
Retrieved 22 titles from url: http://gawker.com/?startTime=1428868800563
Retrieved 20 titles from url: http://gawker.com/?startTime=1428765300817
Retrieved 21 titles from url: http://gawker.com/?startTime=1428686220452
Re

In [4]:
with open('gawker_titles.txt', 'wb') as out:
    out.writelines(gawker_titles)
# with open('gawker_titles.txt') as f:
#     gawker_titles = f.readlines()
    
print "Holy smokes, we got {} Gawker headlines!".format(len(gawker_titles))

Holy smokes, we got 2017 Gawker headlines!


#### [Wall Street Journal](http://online.wsj.com/public/page/archive-2014-1-1.html)
Now we'll do a similar thing with WSJ now. Here we notice that they have a section of the site where they have lists of articles for each day in the past year. There are links to the different archive dates all over the page, and we can see that the links all have the same structure, with different dates in the URL. Lets iterate over a bunch of dates. I grabbed the articles from the first day of each month this year

In [ ]:
wsj_url = "http://online.wsj.com/public/page/archive-2014-{}-1.html"
wsj_headline_xpath = "//h2/a/text()"
wsj_headlines = []
for i in range(1, 11): 
    dom = html.parse(wsj_url.format(i))
    titles = dom.xpath(wsj_headline_xpath)
    wsj_headlines += titles
    print "Retrieved {} WSJ headlines from url: {}".format(len(titles), wsj_url.format(i))
    
    

In [ ]:
with open('wsj_titles.txt', 'wb') as out:
    out.writelines(wsj_headlines)
# with open('wsj_titles.txt') as f:
#     wsj_headlines = f.readlines()
    
print "Jeez, Louise! We got {} WSJ headlines!".format(len(wsj_headlines))

Now we'll use pandas to build a data frame with two columns: "gawker", which contains a boolean value indicating whether the value in the "title" column came from Gawker's website. 

In [ ]:
import pandas as pd
gawk_records = [{'gawker': True, 'title': title} for title in gawker_titles]
wsj_records = [{'gawker': False, 'title': title} for title in wsj_headlines]
df = pd.DataFrame.from_records(gawk_records + wsj_records)
df.tail()

## Teach the Machine!

The basic goal of machine learning is to learn a model or function that maps our inputs/observations to outputs/predictions. The model that we'll be building to make our predictions is known as Naive Bayes. We can compute by counting the probability that any given word shows up in a Gawker title, or P(Word | Gawker). We want the probability of a that a given body of text is a Gawker title, or P(Gawker | Words).

$$P(Gawker | Words) = P(Word_1|Gawker)*P(Word_2|Gawker)*..*P(Word_n | Gawker)*P(Gawker)$$

All of the probabilities on the right side of the equation are empirically determined from the text data. Luckily, instead of having to write all of the code to determine those probabilities ourselves, sklearn's CountVectorizer does all of the dirty work for us.

In [ ]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import cross_validation, naive_bayes

In [ ]:
# def make_xy(df, vectorizer=None):
#     #Your code here    
#     if vectorizer is None:
#         vectorizer = CountVectorizer(min_df=5, max_df=.3, ngram_range=(1,2))
#     X = vectorizer.fit_transform(df.title)
#     X = X.tocsc()  # some versions of sklearn return COO format
#     Y = df.gawker.values.astype(np.int)
#     return X, Y

In [ ]:
vectorizer = CountVectorizer(min_df=5, max_df=.3, ngram_range=(1,2))
# X, Y = make_xy(df)
X = vectorizer.fit_transform(df.title)
X = X.tocsc()  # some versions of sklearn return COO format
Y = df.gawker.values.astype(np.int) # Need numbers instead of bools for 

Now we have a data array, X, whose rows correspond to titles and whose columns correspond to words. So the value X[i, j] is the number of times word j shows up in article title i. Each row has a corresponding member in vector, Y. If the headline associated with X[i] came from Gawker, Y[i] == 1. Otherwise, Y[i] == 0. Now our data are in a format we want it to be in. Time to seperate our data into training and testing sets before building and evaluating our model.

In [ ]:
X_train, X_test, Y_train, Y_test = cross_validation.train_test_split(X,Y)

In [ ]:
clf = naive_bayes.MultinomialNB(fit_prior=False, alpha=0.5)
clf.fit(X_train, Y_train)

## Test the machine!

We'll see how good our model is by testing the accuracy of its predictions on articles it hasn't seen before. The accuracy metric reported below is simply the percentage of titles it classifies correctly.

In [ ]:
print "Accuracy: %0.2f%%" % (100 * clf.score(X_test, Y_test))

Not bad for a stupid machine! We can take a closer look at how the model is making predictions. Let's look at which words are the Gawkeriest and which ones are the Wall Street Journaliest.

In [ ]:
import numpy as np
words = np.array(vectorizer.get_feature_names())

x = np.eye(X_test.shape[1])
probs = clf.predict_log_proba(x)[:, 0]
ind = np.argsort(probs)

good_words = words[ind[:10]]
bad_words = words[ind[-10:]]

good_prob = probs[ind[:10]]
bad_prob = probs[ind[-10:]]

print "Gawker words\t P(gawker| word)"
for w, p in zip(good_words, good_prob):
    print "%20s" % w, "%0.2f" % (1 - np.exp(p))
    
print "WSJ words\t P(gawker | word)"
for w, p in zip(bad_words, bad_prob):
    print "%20s" % w, "%0.2f" % (1 - np.exp(p))

You might be complaining that our model should do better. But some titles can be tough. Would you have classified these mis-predicted article titles correctly? 

In [ ]:
prob = clf.predict_proba(X)[:, 0]
predict = clf.predict(X)

bad_wsj = np.argsort(prob[Y == 0])[:5]
bad_gawker = np.argsort(prob[Y == 1])[-5:]

print "Mis-predicted WSJ quotes"
print '---------------------------'
for row in bad_wsj:
    print df[Y == 0].title.irow(row)
    print

print
print "Mis-predicted Gawker quotes"
print '--------------------------'
for row in bad_gawker:
    print df[Y == 1].title.irow(row)
    print

Make your own headline and see if it belongs in Gawker or WSJ!

In [ ]:
probs = clf.predict_proba(vectorizer.transform(["Your fucking title here"]))
print "P(gawker) = {}".format(probs[0][1])